# 잠재의미분석 (LSA)

## 1.1 실습 템플릿

In [2]:
class LSA:
    def __init__(self, doc_ls, topic_num):
        pass
    
    # tdm matrix 생성
    def TDM(self, doc_ls):
        pass
    
    # tdm matrix 특이값 분해(SVD)
    # U, s, Vt 로 분해
    def SVD(self, tdm):
        pass
    
     # 토픽별 주요 키워드 출력
  def TopicModeling(self) :
    pass
  
  # 단어 벡터 행렬 생성 dot(U,s)  
  def TermVectorMatrix(self, u, s):
    pass
  
  # 문서 벡터 행렬 생성 dot(s,Vt).T 
  def DocVectorMatrix(self, s, vt):
    pass
  
  # 키워드를 입력했을 때 단어 벡터 반환
  def GetTermVector(self, term):
    pass
  
  # 문서를 입력했을 때 문서 벡터 반환
  def GetDocVector(self, doc):
    pass
  
  # 단어-문서 벡터 행렬 생성
  def TermDocVectorMatrix(self, u, s, vt):
    pass
  
  # 단어 벡터 행렬에서 단어 간 코사인 유사도 측정하여 행렬형태로 반환
  def TermSimilarityMatrix(self, term_vec_matrix):
    pass
  
  # 두개 단어를 입력했을 때 코사인 유사도 반환
  def GetTermSimilarity(self, term1, term2):
    pass
  
  # 문서 벡터 행렬에서 문서 간 코사인 유사도 측정하여 행렬형태로 반환
  def DocSimilarityMartrix(self, doc_vec_matrix):
    pass
  
  # 두개 문서를 입력했을 때 코사인 유사도 반환
  def GetDocSimilarity(self, doc1, doc2):
    pass

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [3]:
doc_ls = ['바나나 사과 포도 포도',
         '사과 포도',
         '포도 바나나',
         '짜장면 짬뽕 탕수욕',
         '볶음밥 탕수욕',
         '짜장면 짬뽕',
         '라면 스시',
         '스시',
         '가츠동 스시 소바',
         '된장찌개 김치찌개 김치',
         '김치 된장',
         '비빔밥 김치'
         ]

lsa = LSA(doc_ls, 3)
lsa.TopicModeling()
lsa.GetTermSimilarity('사과','바나나')
lsa.GetTermSimilarity('사과','짜장면')
lsa.GetDocSimilarity('사과 포도', '포도 바나나')
lsa.GetDocSimilarity('사과 포도', '라면 스시')

NameError: name 'LSA' is not defined

## 1.2 실습 예제 코드

In [7]:
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import randomized_svd

class LSA :
    def __init__(self, doc_ls, topic_num):
        self.doc_ls = doc_ls
        self.topic_num = topic_num
        self.term2idx, self.idx2term = self.toIdxDict(' '.join(doc_ls).split())
        self.doc2idx, self.idx2doc = self.toIdxDict(doc_ls)

        self.tdm = self.TDM(doc_ls)
        self.U, self.s, self.VT = self.SVD(self.tdm)

        #print(self.s)
        #print(self.U[:,:1].round(2))

        self.term_mat = self.TermVectorMatrix(self.U, self.s, topic_num)
        self.doc_mat = self.DocVectorMatrix(self.s, self.VT, topic_num)
        self.term_doc_mat = self.TermDocVectorMatrix(self.U, self.s, self.VT, topic_num)

        #print(self.term2idx.keys())
        #print(self.term_mat.round(2))

        self.term_sim = self.TermSimilarityMatrix(self.term_mat)
        self.doc_sim = self.DocSimilarityMartrix(self.doc_mat)

    # 리스트내 값을 index로 변환하는 dict과 
    # index를 리스트내 값으로 변환하는 dict
    def toIdxDict(self, ls) :
        any2idx = defaultdict(lambda : len(any2idx))
        idx2any = defaultdict()

        for item in ls:
            any2idx[item]
            idx2any[any2idx[item]] = item

        return any2idx, idx2any

    def TDM(self, doc_ls):
        # 행(토큰크기), 열(문서갯수)로 TDM 생성
        tdm = np.zeros([len(self.term2idx.keys()), len(doc_ls)])

        for doc_idx, doc in enumerate(doc_ls) :
            for term in doc.split() :
            #등장한 단어를 dictionary에서 위치를 탐색하여 빈도수 세기
                tdm[self.term2idx[term], doc_idx] += 1
    
        return tdm

    # 특이값 분해
    def SVD(self, tdm):
        U, s, VT = randomized_svd(tdm, 
                          n_components=15,
                          n_iter=5,
                          random_state=None)

        #U, s, VT = np.linalg.svd(tdm)
        return U, s, VT

    # 토픽별 주요 키워드 출력
    def TopicModeling(self, count = 3) :
        topic_num = self.topic_num

        for i in range(topic_num) :
            score = self.U[:,i:i+1].T
            sorted_index = np.flip(np.argsort(-score),0)

            a = []
            for j in sorted_index[0,: count] :
                a.append((self.idx2term[j], score[0,j].round(3)))

        print("Topic {} - {}".format(i+1,a ))

    def vectorSimilarity(self, matrix) :
        similarity = np.zeros([matrix.shape[1], matrix.shape[1]])

        for i in range(matrix.shape[1]) :
            for j in range(matrix.shape[1]) :
                similarity[i,j] =  cosine_similarity(matrix[:,i].T, matrix[:,j].T)

        return similarity

    # 키워드를 입력했을 때 단어 벡터 반환
    def GetTermVector(self, term):
        vec = self.term_mat[self.term2idx[term]:self.term2idx[term]+1,:]
        print('{} = {}'.format(term, vec))
        return vec

    # 문서를 입력했을 때 문서 벡터 반환
    def GetDocVector(self, doc):
        vec = self.doc_mat.T[self.doc2idx[doc]:self.doc2idx[doc]+1,:]
        print('{} = {}'.format(doc, vec))
        return vec

    def Compression(self, round_num=0) :
        print(self.tdm)
        print(self.term_doc_mat.round(round_num))

    def TermVectorMatrix(self, u, s, topic_num):
        term_mat = np.matrix(u[:, :topic_num])# * np.diag(s[:topic_num])
        return term_mat

    def DocVectorMatrix(self, s, vt, topic_num):
        doc_mat = np.diag(s[:topic_num]) * np.matrix(vt[:topic_num,:])
        return doc_mat

    def TermDocVectorMatrix(self, u, s, vt, topic_num):
        term_doc_mat = np.matrix(u[:, :topic_num]) * np.diag(s[:topic_num])  * np.matrix(vt[:topic_num,:])
        return term_doc_mat

    def TermSimilarityMatrix(self, termVectorMatrix):
        return self.vectorSimilarity(termVectorMatrix.T)

    def GetTermSimilarity(self, term1, term2):
        sim = self.term_sim[self.term2idx[term1], self.term2idx[term2]]
        print("({},{}) term similarity = {}".format(term1, term2, sim))
        return sim 

    def DocSimilarityMartrix(self,docVectorMatrix):    
        return self.vectorSimilarity(docVectorMatrix) 

    def GetDocSimilarity(self, doc1, doc2):
        sim = self.doc_sim[self.doc2idx[doc1], self.doc2idx[doc2]]
        print("('{}','{}') doc similarity = {}".format(doc1, doc2, sim))
        return sim 

In [8]:
doc_ls = ['바나나 사과 포도 포도 짜장면',
         '사과 포도',
         '포도 바나나',
         '짜장면 짬뽕 탕수육',
         '볶음밥 탕수육',
         '짜장면 짬뽕',
         '라면 스시',
         '스시 짜장면',
         '가츠동 스시 소바',
         '된장찌개 김치찌개 김치',
         '김치 된장 짜장면',
         '비빔밥 김치'
         ]

lsa = LSA(doc_ls, 4)
X = lsa.TDM(doc_ls)
print('== 토픽 모델링 ==')
lsa.TopicModeling(4)
print('\n== 단어 벡터 ==')
lsa.GetTermVector('사과')
lsa.GetTermVector('짜장면')
print('\n== 단어 유사도 ==')
lsa.GetTermSimilarity('사과','바나나')
lsa.GetTermSimilarity('사과','짜장면')
lsa.GetTermSimilarity('포도','짜장면')
lsa.GetTermSimilarity('사과','스시')
print('\n== 문서 벡터 ==')
lsa.GetDocVector('사과 포도')
lsa.GetDocVector('짜장면 짬뽕')
print('\n== 문서 유사도 ==')
lsa.GetDocSimilarity('사과 포도', '포도 바나나')
lsa.GetDocSimilarity('사과 포도', '라면 스시')
print('\n== 토픽 차원수로 압축 ==')
lsa.Compression(0)

== 토픽 모델링 ==
Topic 4 - [('스시', 0.552), ('김치', 0.371), ('가츠동', 0.277), ('소바', 0.277)]

== 단어 벡터 ==
사과 = [[ 0.34843127 -0.19370961  0.01592593  0.03744775]]
짜장면 = [[ 0.48563449  0.58415588 -0.07468389 -0.18737521]]

== 단어 유사도 ==
(사과,바나나) term similarity = 1.0
(사과,짜장면) term similarity = 0.15191335214807028
(포도,짜장면) term similarity = 0.15191335214807003
(사과,스시) term similarity = -0.04097825202732763

== 문서 벡터 ==
사과 포도 = [[ 1.04529381 -0.58112882  0.04777778  0.11234324]]
짜장면 짬뽕 = [[ 0.61011838  0.93971158 -0.17760018 -0.53682795]]

== 문서 유사도 ==
('사과 포도','포도 바나나') doc similarity = 1.0
('사과 포도','라면 스시') doc similarity = -0.038506882113500146

== 토픽 차원수로 압축 ==
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 

## 1.3 sklearn LSA 구현

In [2]:
doc_ls = ['바나나 사과 포도 포도',
         '사과 포도',
         '포도 바나나',
         '짜장면 짬뽕 탕수욕',
         '볶음밥 탕수욕',
         '짜장면 짬뽕',
         '라면 스시',
         '스시',
         '가츠동 스시 소바',
         '된장찌개 김치찌개 김치',
         '김치 된장',
         '비빔밥 김치'
         ]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(doc_ls)
print(X)

from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=4, algorithm='randomized', n_iter=100)
svd_model.fit(X)

np.shape(svd_model.components_)

  (0, 6)	0.4418867148991048
  (0, 9)	0.4418867148991048
  (0, 15)	0.7806870451028085
  (1, 9)	0.7494645793884686
  (1, 15)	0.662044442799776
  (2, 6)	0.7494645793884686
  (2, 15)	0.662044442799776
  (3, 12)	0.5773502691896258
  (3, 13)	0.5773502691896258
  (3, 14)	0.5773502691896258
  (4, 14)	0.6515208683486948
  (4, 7)	0.7586307126040724
  (5, 12)	0.7071067811865476
  (5, 13)	0.7071067811865476
  (6, 5)	0.7966850663542188
  (6, 11)	0.6043946600096447
  (7, 11)	1.0
  (8, 11)	0.47271637301705055
  (8, 0)	0.6231128431839633
  (8, 10)	0.6231128431839633
  (9, 4)	0.6231128431839633
  (9, 2)	0.6231128431839633
  (9, 1)	0.47271637301705055
  (10, 1)	0.6043946600096447
  (10, 3)	0.7966850663542188
  (11, 1)	0.6043946600096447
  (11, 8)	0.7966850663542188


(4, 16)

In [5]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=3):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('포도', 0.78069), ('바나나', 0.44189), ('사과', 0.44189)]
Topic 2: [('스시', 0.8864), ('라면', 0.33189), ('소바', 0.2282)]
Topic 3: [('짬뽕', 0.6258), ('짜장면', 0.6258), ('탕수욕', 0.43614)]
Topic 4: [('김치', 0.76421), ('된장', 0.37119), ('비빔밥', 0.37119)]


# 2. LDA 실습

## 2.1 실습 템플릿

In [12]:
class LDA :
    def __init__(self, doc_ls, topic_num, alpha = 0.1, beta = 0.001):
        self.alpha = alpha
        self.beta = beta
        self.k = topic_num

    def RandomlyAssignTopic(self, doc_ls):
        pass


    def IterateAssignTopic(self) :
        pass

    # 토픽별 주요 키워드 출력
    def TopicModeling(self) :
        pass


## 2.2 실습 예제 코드

In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
import random
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
from collections import defaultdict



class LDA :
    def __init__(self, docs, topic_num, alpha = 0.1, beta = 0.001):
        self.alpha = alpha
        self.beta = beta
        self.k = topic_num
        self.docs = docs
        
        
    def RandomlyAssignTopic(self, docs) :
        dic = defaultdict()
        t2i = defaultdict(lambda : len(t2i))
        i2t = defaultdict()
        d = 0
        w = 0
    
        wnl = WordNetLemmatizer()
        stopword = stopwords.words('english')
        stopword.append(',')

        # 임의의 토픽을 할당
        for tokens in [word_tokenize(doc) for doc in docs] :
            for token in [wnl.lemmatize(token.lower()) for token in tokens 
                        if token not in stopword] :
                            i2t[t2i[token]] = token
                            dic[(d, t2i[token], w)] = random.randint(0,self.k-1)
                            w += 1
                            d += 1
        
        print(dic)
        return dic, t2i, i2t

    
    def CountDocTopic(self, t2i) :
        docs = np.zeros((self.k, len(self.docs)))
        terms = np.zeros((self.k, len(t2i.keys())))


        #문서별 토큰별 빈도수 계산
        for (d, n, w) in self.term_topic.keys() :
            topic = self.term_topic[(d, n, w)]
            docs[topic, d] += 1 + self.alpha
            terms[topic, n] += 1 + self.beta

        #비어있는 값는 값에 alpha, beta 설정
        docs = np.where(docs==0.0, self.alpha, docs) 
        terms = np.where(terms==0.0, self.beta, terms)

        print("단어 토픽별 빈도")
        print(terms.round(2))
        print("문서 토픽별 빈도")
        print(docs.round(1))

        return docs, terms

    
    def IterateAssignTopic(self, docs, terms, i2t) :
    #한개의 단어씩 주제 배정
        prev = {}
    
        while prev != self.term_topic:
            for (d, n, w) in self.term_topic.keys() :
                topic = [0, 0]

                docs[self.term_topic[(d, n, w)], d] -= (1 + self.alpha)
                terms[self.term_topic[(d, n, w)], n] -= (1 + self.beta)

                docs = np.where(docs==0.0, self.alpha, docs) 
                terms = np.where(terms==0.0, self.beta, terms)

                print()
                print("{}(d:{}, n:{}, w:{}) = topic:{}".format(i2t[n], d, n, w, self.term_topic[(d, n, w)]))
                print("문서 토픽별 빈도")
                print(docs.round(1))
                print("단어 토픽별 빈도")
                print(terms.round(3))


                prev = self.term_topic

                for t in range(self.k) :
                    p_t_d = docs[t, d]/docs[:,d:d+1].sum()
                    p_w_t = terms[t, n]/terms[t:t+1,:].sum()
                    prob = p_t_d * p_w_t

                    if topic[1] < prob :
                        topic = [t, prob]

                print("topic {} 일 확률 = {}/{} * {}/{} = {} * {} = {}".format(t
                    ,docs[t, d].round(1) , docs[:,d:d+1].sum().round(1)
                    ,terms[t, d].round(3) , terms[t:t+1,:].sum().round(3)
                    , p_t_d.round(3), p_w_t.round(4), prob.round(4)))


            if docs[topic[0], d] < 1 : docs[topic[0], d] = 0;
            if terms[topic[0], n] < 1 : terms[topic[0], n] = 0;

            #확률이 가장 큰 토픽을 할당  
            self.term_topic[(d, n, w)] = topic[0]
            docs[topic[0], d] += (1 + self.alpha)
            terms[topic[0], n] += (1 + self.beta)

            print("할당된 토픽:{}".format(self.term_topic[(d, n, w)]))
            print("="*50)

        return terms

  # 토픽별 주요 키워드 출력
    def TopicModeling(self, count=3) :
        self.term_topic, t2i, i2t = self.RandomlyAssignTopic(self.docs)
        docs, terms = self.CountDocTopic(t2i)
        terms = self.IterateAssignTopic(docs, terms, i2t)

        score = terms / terms.sum(axis=1, keepdims=True)

        for i in range(self.k) :
            print("\nTopic {}".format(i+1))
            sorted_index = np.flip(np.argsort(score[i]),0)[:count]
            for j in sorted_index :
              #pass
              print("({}={})".format(i2t[j], score[i,j].round(3)), end = ' ')


In [27]:
doc_ls = ["Cute kitty",
"Eat rice or cake",
"Kitty and hamster",
"Eat bread",
"Rice, bread and cake",
"Cute hamster eats bread and cake"]

lda = LDA(doc_ls, 2)
lda.TopicModeling(5)

defaultdict(None, {(0, 0, 0): 1, (1, 1, 1): 0, (2, 2, 2): 1, (3, 3, 3): 0, (4, 4, 4): 0, (5, 1, 5): 0, (6, 5, 6): 1, (7, 2, 7): 0, (8, 6, 8): 1, (9, 3, 9): 0, (10, 6, 10): 1, (11, 4, 11): 0, (12, 0, 12): 0, (13, 5, 13): 0, (14, 7, 14): 1, (15, 6, 15): 0, (16, 4, 16): 1})


IndexError: index 6 is out of bounds for axis 1 with size 6